This notebook is based on the tutorial in Object Detection API and should run with Tensorflow 2.0, assuming that the Tensorflow Object Detection API is installed.

This notebook assumes 
1. that images with trafic lights are stored in a directory called 'raw_images_simultion' organized into 4 different subdirectories wccording to the colors of the lights:

    raw_images_simulation$ ls
    
    green  none  red  yellow

and does the following:
1. Uses a pretrained model and find boxes with potential trafic lights. As the images are already ordered in sub directories, the valid color of the light is known.

2. Goes threough all images and boxes and checks that the found box has a reasonable score for being a traffic light, has a reasonable size and aspect ratio and actually contains a trafic light or correct color. In that case the image and the box are taken and are labeled.

Please start by setting these two path variables so that the remaining parts work properly:

In [1]:
# Please set these two paths
path_to_tensorflow_models = '/home/said/ML_tools/tensorflow37/models'
path_to_tl_detection = '/home/said/GIT/CarND-Capstone/tl_detection_pipeline'

import os
# We run the code from that directory:
os.chdir(path_to_tl_detection)

In [2]:
import pathlib

import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import numpy as np

from collections import defaultdict
from io import StringIO
import tkinter
from matplotlib import pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

from PIL import Image
from IPython.display import display

# Used for image processing and manipulations
import cv2

# Used for internal storage
import pickle

# Used for data augmentation
from skimage import exposure
from skimage import util

# Used for dividing the data into training and testing
import pandas as pd
from random import seed
from random import random

Import the object detection module.

In [3]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [4]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [5]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"
  print(model_dir)

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

## Loading the model and label map for initial detection
The object initial detection model is assumed to exist and will be loaded here. 

First, these two paths must be set properly:

In [6]:
PATH_TO_LABELS = path_to_tensorflow_models + '/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
model_name = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
detection_model = load_model(model_name)

PATH_TO_TEST_IMAGES_DIR = pathlib.Path('./raw-images-simulation')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*/*.jpg")))

/home/said/.keras/datasets/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03/saved_model
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [7]:
print("detection_model.inputs: ")
print(detection_model.inputs, "\n")
print("detection_model.output_dtypes: ")
print(detection_model.output_dtypes, "\n")
print("detection_model.output_shapes: ")
print(detection_model.output_shapes, "\n")

detection_model.inputs: 
[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>] 

detection_model.output_dtypes: 
{'detection_classes': tf.float32, 'num_detections': tf.float32, 'detection_boxes': tf.float32, 'detection_scores': tf.float32} 

detection_model.output_shapes: 
{'detection_classes': TensorShape([None, 100]), 'num_detections': TensorShape([None]), 'detection_boxes': TensorShape([None, 100, 4]), 'detection_scores': TensorShape([None, 100])} 



Add a wrapper function to call the model, and cleanup the outputs:

In [8]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [9]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  output_dict['file_name'] = str(image_path).split('/')[-1]
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  # display(Image.fromarray(image_np))
  return output_dict, Image.fromarray(image_np)

### First step of annotations
Go through all images and make a list of annotation objects. Save the list as a pickle file so that it can be reused.

In [10]:
rows = len(TEST_IMAGE_PATHS) // 5 + 1
fig = plt.figure(figsize=(12,3*rows))
annotations = []
counter = 0
for image_path in TEST_IMAGE_PATHS:
  counter += 1
  print('Number {} out of {}, file name: {}'.format(counter, len(TEST_IMAGE_PATHS), image_path))
  output_dict, boxed_image = show_inference(detection_model, image_path)
  color = str(image_path).split('/')[-2]
  output_dict['color'] = color
  annotations.append(output_dict)
  ax = plt.subplot(rows,5,len(annotations))
  ax.imshow(boxed_image)
  ax.set_title("{}: {}".format(color, output_dict['num_detections']), fontsize=10)
  ax.axis("off")
plt.show()

pickle.dump(annotations, open(path_to_tl_detection + '/annotations/' + model_name + '.p', 'wb')) 


Number 1 out of 131, file name: raw-images-simulation/green/left0540.jpg
Number 2 out of 131, file name: raw-images-simulation/green/left0541.jpg
Number 3 out of 131, file name: raw-images-simulation/green/left0542.jpg
Number 4 out of 131, file name: raw-images-simulation/green/left0543.jpg
Number 5 out of 131, file name: raw-images-simulation/green/left0544.jpg
Number 6 out of 131, file name: raw-images-simulation/green/left0545.jpg
Number 7 out of 131, file name: raw-images-simulation/green/left0546.jpg
Number 8 out of 131, file name: raw-images-simulation/green/left0547.jpg
Number 9 out of 131, file name: raw-images-simulation/green/left0548.jpg
Number 10 out of 131, file name: raw-images-simulation/green/left0549.jpg
Number 11 out of 131, file name: raw-images-simulation/green/left0550.jpg
Number 12 out of 131, file name: raw-images-simulation/green/left0551.jpg
Number 13 out of 131, file name: raw-images-simulation/green/left0552.jpg
Number 14 out of 131, file name: raw-images-sim

Number 114 out of 131, file name: raw-images-simulation/yellow/left0021.jpg
Number 115 out of 131, file name: raw-images-simulation/yellow/left0022.jpg
Number 116 out of 131, file name: raw-images-simulation/yellow/left0570.jpg
Number 117 out of 131, file name: raw-images-simulation/yellow/left0571.jpg
Number 118 out of 131, file name: raw-images-simulation/yellow/left0572.jpg
Number 119 out of 131, file name: raw-images-simulation/yellow/left0574.jpg
Number 120 out of 131, file name: raw-images-simulation/yellow/left0575.jpg
Number 121 out of 131, file name: raw-images-simulation/yellow/left0576.jpg
Number 122 out of 131, file name: raw-images-simulation/yellow/left0577.jpg
Number 123 out of 131, file name: raw-images-simulation/yellow/left0579.jpg
Number 124 out of 131, file name: raw-images-simulation/yellow/left0580.jpg
Number 125 out of 131, file name: raw-images-simulation/yellow/left0581.jpg
Number 126 out of 131, file name: raw-images-simulation/yellow/left0582.jpg
Number 127 o

/home/said/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  app.launch_new_instance()


## Filtering and augmentation
Define the filter function and filter the annotated boxes so that correct boxes are remaining. After filtering, data should be augmented. For that, further image processing steps are defined.

In [11]:
def filter_annotations(annotation, image_path, show_image=False):
    #print(annotation)
    # [y_min, x_min, y_max, x_max]
    output={}
    output["file_name"] = image_path + "/" + annotation["color"] + "/" + annotation["file_name"]
    output["color"] = annotation["color"]
    output["boxes"] =[]
    output["flipped_boxes"] =[]
    image = plt.imread(output["file_name"])
    flipped_image = cv2.flip(image, 1)

    y_scale = image.shape[0]
    x_scale = image.shape[1]

    boxed_image = image
    boxed_flipped_image = flipped_image
    for i in range(len(annotation["detection_boxes"])):
        box = annotation["detection_boxes"][i]
        start_point = (int(x_scale*box[1]), int(y_scale*box[0]))
        end_point = (int(x_scale*box[3]), int(y_scale*box[2]))
        flipped_start_point = (int(x_scale*(1-box[1])), int(y_scale*box[0]))
        flipped_end_point = (int(x_scale*(1-box[3])), int(y_scale*box[2]))
        height = end_point[1] - start_point[1]
        width  = end_point[0] - start_point[0]
        img = image[start_point[1]:end_point[1], start_point[0]:end_point[0], :]
        imshape = img.shape
        imsize = imshape[0] * imshape[1]
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        
        # color range
        lower_red1 = np.array([0,100,100])
        upper_red1 = np.array([10,255,255])
        lower_red2 = np.array([160,100,100])
        upper_red2 = np.array([180,255,255])
        lower_green = np.array([40,50,50])
        upper_green = np.array([90,255,255])
        # lower_yellow = np.array([15,100,100])
        # upper_yellow = np.array([35,255,255])
        lower_yellow = np.array([15,150,150])
        upper_yellow = np.array([35,255,255])
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        maskg = cv2.inRange(hsv, lower_green, upper_green)
        masky = cv2.inRange(hsv, lower_yellow, upper_yellow)
        maskr = cv2.add(mask1, mask2)
        
        #print("Red: {}, Green: {}, Yellow: {}".format(maskr.shape, maskg.shape, masky.shape))
        #print("Size: {}, Red: {}, Green: {}, Yellow: {}".format(imsize, (maskr>0).sum(), (maskg>0).sum(), (masky>0).sum()))
        
        r_size = (maskr>0).sum() # ((maskr>0) * (maskg==0) * (masky == 0)).sum()
        g_size = (maskg>0).sum()
        y_size = (masky>0).sum()
        
        most_color = ""
        if (r_size > max(imsize/50, g_size, y_size)):
            most_color = 'red'
        elif (g_size > max(imsize/50, r_size, y_size)):
            most_color = 'green'
        elif (y_size > max(imsize/50, g_size, r_size)):
            most_color = 'yellow'

        aspect_ratio = height/width
        x_ratio = width / x_scale
        y_ratio = height / y_scale
        if (aspect_ratio > 1.5) and (aspect_ratio < 10) and (annotation['detection_scores'][i] > 0.2) and \
            (x_ratio < 1/6) and (y_ratio < 1/2) :
            if ((most_color == annotation["color"]) or (len(annotation["detection_boxes"]) == 1)):
                #print("Aspect ratio: {}, x_ratio: {}, y_ratio: {}, Score: {}".format(aspect_ratio, x_ratio, y_ratio, annotation['detection_scores'][i]))
        
                output["boxes"].append([start_point,end_point])
                output["flipped_boxes"].append([flipped_start_point,flipped_end_point])
                boxed_image = cv2.rectangle(boxed_image,start_point,end_point,(0,255,255),4)
                boxed_flipped_image = cv2.rectangle(boxed_flipped_image,flipped_start_point,flipped_end_point,(0,255,255),4)
    if (show_image):
        fig = plt.figure(figsize=(12,4))
        ax = plt.subplot(1,2,1)
        ax.imshow(boxed_image)   
        ax.axis("off")
        ax = plt.subplot(1,2,2)
        ax.imshow(boxed_flipped_image)
        ax.axis("off")
        plt.show()
    return output


In [12]:

def add_brightness(image):    
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS    
    image_HLS = np.array(image_HLS, dtype = np.float64)     
    random_brightness_coefficient = np.random.uniform()+0.5 ## generates value between 0.5 and 1.5    
    brightness_coefficient = 1.2
    image_HLS[:,:,1] = image_HLS[:,:,1]*brightness_coefficient ## scale pixel values up or down for channel 1(Lightness)    
    image_HLS[:,:,1][image_HLS[:,:,1]>255]  = 255 ##Sets all values above 255 to 255    
    image_HLS = np.array(image_HLS, dtype = np.uint8)    
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB    
    return image_RGB

def generate_random_lines(imshape,slant,drop_length):    
    drops=[]    
    for i in range(1500): ## If You want heavy rain, try increasing this        
        if slant<0:            
            x= np.random.randint(slant,imshape[1])        
        else:            
            x= np.random.randint(0,imshape[1]-slant)        
        y= np.random.randint(0,imshape[0]-drop_length)        
        drops.append((x,y))    
    return drops            

def add_rain(image):        
    image = image.copy()
    imshape = image.shape    
    slant_extreme=10    
    slant= np.random.randint(-slant_extreme,slant_extreme)     
    drop_length=20    
    drop_width=2 
    drop_color=(200,200,200) 
    ## a shade of gray    
    rain_drops= generate_random_lines(imshape,slant,drop_length)        
    for rain_drop in rain_drops:        
        cv2.line(image,(rain_drop[0],rain_drop[1]),(rain_drop[0]+slant,rain_drop[1]+drop_length),drop_color,drop_width)    
    image= cv2.blur(image,(7,7)) ## rainy view are blurry        
    brightness_coefficient = 0.7 ## rainy days are usually shady     
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS    
    image_HLS[:,:,1] = image_HLS[:,:,1]*brightness_coefficient ## scale pixel values down for channel 1(Lightness)    
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB    
    return image_RGB

def generate_shadow_coordinates(imshape, no_of_shadows=1):    
    vertices_list=[]    
    for index in range(no_of_shadows):        
        vertex=[]        
        for dimensions in range(np.random.randint(3,15)): ## Dimensionality of the shadow polygon            
            vertex.append(( imshape[1]*np.random.uniform(),imshape[0]//3+imshape[0]*np.random.uniform()))        
            vertices = np.array([vertex], dtype=np.int32) ## single shadow vertices         
            vertices_list.append(vertices)
    return vertices_list ## List of shadow vertices


def add_shadow(image,no_of_shadows=1):    
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS    
    mask = np.zeros_like(image)     
    imshape = image.shape    
    vertices_list= generate_shadow_coordinates(imshape, no_of_shadows) #3 getting list of shadow vertices    
    for vertices in vertices_list:         
        cv2.fillPoly(mask, vertices, 255) ## adding all shadow polygons on empty mask, single 255 denotes only red channel        
    image_HLS[:,:,1][mask[:,:,0]==255] = image_HLS[:,:,1][mask[:,:,0]==255]*7   ## if red channel is hot, image's "Lightness" channel's brightness is lowered     
    image_HLS[:,:,1][image_HLS[:,:,1]>255]  = 255 ##Sets all values above 255 to 255 
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB    
    return image_RGB

def augmentData(original_image):
    
    blured_image = cv2.GaussianBlur(original_image,(15,15),0)
    image_with_random_noise = np.uint8(util.random_noise(original_image)*255)
    image_with_random_noise[image_with_random_noise > 255]  = 255
    image_with_random_noise[image_with_random_noise < 0]  = 0
    image_with_random_noise = np.uint8(image_with_random_noise)
    v_min, v_max = np.percentile(original_image, (0.2, 99.8))
    better_contrast = exposure.rescale_intensity(original_image, in_range=(v_min, v_max))
    bright_image = add_brightness(original_image)
    rainy_image = add_rain(original_image)
    shadowed_image = add_shadow(original_image,no_of_shadows=1)
    
    return [original_image, blured_image, image_with_random_noise, bright_image, rainy_image]


Read the annotation list and filter it.

In [13]:
read_annotations = pickle.load(open(path_to_tl_detection + '/annotations/' + model_name + '.p', 'rb'))
filtered_annotations = []
for i in range(0,len(read_annotations)):
    filtered_annotations.append(filter_annotations(read_annotations[i], str(PATH_TO_TEST_IMAGES_DIR),show_image=False))

### Divide the data into training and test

In [14]:
def select_list(probability, choice, alternative_choice):
    if (random() <= probability):
        return choice
    else:
        return alternative_choice

def make_cvs_file(annotations, output_path, test_partition = 0.15):
    annotation_lists = {}
    annotation_lists["train"] = []
    annotation_lists["test"] = []
    counter = 0
    for i in range(len(annotations)):
        counter += 1
        fname = str(counter).zfill(5)
        
        image = plt.imread(annotations[i]["file_name"])
        
        width = image.shape[1]
        height = image.shape[0]
        class_name = annotations[i]["color"]
        boxes = annotations[i]["boxes"]
        flipped_boxes = annotations[i]["flipped_boxes"]
        
        if (len(boxes) > 0):
            images = augmentData(image)
            
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_o_rigal" + ".jpg"
            plt.imsave(filename, images[0], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_o_blure" + ".jpg"
            plt.imsave(filename, images[1], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_o_noise" + ".jpg"
            plt.imsave(filename, images[2], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_o_brght" + ".jpg"
            plt.imsave(filename, images[3], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_o_rainy" + ".jpg"
            plt.imsave(filename, images[4], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))

            images = augmentData(cv2.flip(image, 1))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_f_lippd" + ".jpg"
            plt.imsave(filename, images[0], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_f_blure" + ".jpg"
            plt.imsave(filename, images[1], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_f_noise" + ".jpg"
            plt.imsave(filename, images[2], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_f_brght" + ".jpg"
            plt.imsave(filename, images[3], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))
            selection = select_list(test_partition, "test", "train")
            filename = output_path + "/images/" + selection + "/" + fname + "_f_rainy" + ".jpg"
            plt.imsave(filename, images[4], format="jpg")
            for box in boxes:
                annotation_lists[selection].append((filename, width, height, class_name, \
                                          box[0][0], box[0][1], box[1][0], box[1][1]))

    
    column_name = ['filename', 'width', 'height',
                'class', 'xmin', 'ymin', 'xmax', 'ymax']
    training_data_frame = pd.DataFrame(annotation_lists["train"], columns=column_name)
    training_data_frame.to_csv(output_path + "/annotations/train_annotation_list.csv")
    test_data_frame = pd.DataFrame(annotation_lists["test"], columns=column_name)
    test_data_frame.to_csv(output_path + "/annotations/test_annotation_list.csv")
    return test_data_frame, training_data_frame

In [15]:
test_df, training_df = make_cvs_file(filtered_annotations, path_to_tl_detection)

In [16]:
nn=3
NN=1
for i in range(nn):
    images = augmentData(plt.imread(filtered_annotations[NN+i]["file_name"]))
    fig = plt.figure(figsize=(3*len(images),3))
    for j in range(len(images)):
        ax = plt.subplot(1,len(images),j+1)
        ax.imshow(images[j])
        ax.axis("off")
        ax.set_title(filtered_annotations[i]["color"], fontsize=10)
    plt.show()

/home/said/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  # This is added back by InteractiveShellApp.init_path()


In [17]:
%%bash
python scripts/generate_tfrecord_2.py --label=no_label --csv_input=./annotations/train_annotation_list.csv  --output_path=./annotations/train.record
python scripts/generate_tfrecord_2.py --label=no_label --csv_input=./annotations/test_annotation_list.csv  --output_path=./annotations/test.record

Successfully created the TFRecords: /home/said/GIT/CarND-Capstone/tl_detection_pipeline/./annotations/train.record
Successfully created the TFRecords: /home/said/GIT/CarND-Capstone/tl_detection_pipeline/./annotations/test.record


In [18]:
# Here, just a test to see if the original script works as expected

!python ~/ML_tools/tensorflow37/models/research/object_detection/model_main.py --pipeline_config_path=./training/ssd_inception_v2_coco.config --model_dir=./training/ssd_inception_v2_coco.config \
    --num_train_steps=500 --sample_1_of_n_eval_examples=1 --alsologtostderr


Traceback (most recent call last):
  File "/home/said/ML_tools/tensorflow37/models/research/object_detection/model_main.py", line 26, in <module>
    from object_detection import model_lib
  File "/home/said/ML_tools/tensorflow37/models/research/object_detection/model_lib.py", line 27, in <module>
    from object_detection import eval_util
  File "/home/said/ML_tools/tensorflow37/models/research/object_detection/eval_util.py", line 40, in <module>
    slim = tf.contrib.slim
AttributeError: module 'tensorflow' has no attribute 'contrib'
